# OLS via Neural Network

Interpret linear regression as an extremely simple feedforward neural network.


In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import statsmodels.api as sm

# Set random seed for reproducibility
np.random.seed(42)

# Generate synthetic data with three regressors
n = 100  # number of observations
x1 = np.random.normal(0, 1, n)  # first explanatory variable
x2 = np.random.normal(0, 1, n)  # second explanatory variable
x3 = np.random.normal(0, 1, n)  # third explanatory variable
beta_0 = 0.5  # intercept
beta_1 = 1.  # coefficient for x1
beta_2 = -1.  # coefficient for x2
beta_3 = 1.  # coefficient for x3
epsilon = np.random.normal(0, 1, n)  # error term with standard normal distribution
y = beta_0 + beta_1 * x1 + beta_2 * x2 + beta_3 * x3 + epsilon  # dependent variable

# Create a DataFrame
data = pd.DataFrame({'x1': x1, 'x2': x2, 'x3': x3, 'y': y})


C:\Users\zhent\AppData\Local\Temp\ipykernel_41380\3021539496.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Use `sm.OLS()` as in statistics / econometrics.

In [2]:
# Fit OLS regression using statsmodels
# produce an X with `x1`, x2`, and `x3` in data
X = data[['x1', 'x2', 'x3']]
X = sm.add_constant(X)  # add a constant term to the model
y = data['y']

model = sm.OLS(y, X).fit()
print("OLS Regression Results Summary:")
print(model.summary())



OLS Regression Results Summary:
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.817
Model:                            OLS   Adj. R-squared:                  0.811
Method:                 Least Squares   F-statistic:                     142.8
Date:                Fri, 07 Mar 2025   Prob (F-statistic):           2.84e-35
Time:                        15:14:45   Log-Likelihood:                -127.46
No. Observations:                 100   AIC:                             262.9
Df Residuals:                      96   BIC:                             273.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.587

A `pytorch` implemenation.

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

# Convert the data to PyTorch tensors
X_tensor = torch.tensor(data[['x1', 'x2', 'x3']].values, dtype=torch.float32)
y_tensor = torch.tensor(data['y'].values.reshape(-1, 1), dtype=torch.float32)

In [4]:
# Define a simple neural network with one hidden layer
class OLS_NN(nn.Module):
    def __init__(self):
        super(OLS_NN, self).__init__()
        self.output = nn.Linear(3, 1)  # three inputs, one output

    def forward(self, x):
        return self.output(x)  # identity function as activation


In [5]:
# Instantiate the model, define the loss function and the optimizer
model = OLS_NN()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [6]:
# Train the model
epochs = 1000
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 200 == 0:
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}')


Epoch 200/1000, Loss: 0.7511
Epoch 400/1000, Loss: 0.7493
Epoch 600/1000, Loss: 0.7493
Epoch 800/1000, Loss: 0.7493
Epoch 1000/1000, Loss: 0.7493


In [7]:
# Report the weights after training
print("\nWeights after training:")
for name, param in model.named_parameters():
    print(f"{name}: {param.data.numpy()}")


Weights after training:
output.weight: [[ 0.8226739 -1.0382419  1.0269035]]
output.bias: [0.587531]


The estimated coefficients from NN are the same as from OLS.